In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.1 MB/s eta 0:00:00


In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/Job_application/VK_ML_engineer/emailTextsI.csv')

<ipython-input-6-de11ffb664c4>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('drive/MyDrive/Job_application/VK_ML_engineer/emailTextsI.csv')


In [ ]:
df = df[:20000]
df.head()

,id,label,text,introduction
0,0,Spam,viiiiiiagraaaa\nonly for the ones that want to...,NaN
1,1,Ham,got ice thought look az original message ice o...,NaN
2,2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...,NaN
3,3,Spam,start increasing your odds of success & live s...,NaN
4,4,Ham,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...


In [ ]:
mapping = {'Spam': 1, 'Ham': 0}
df = df.replace({'label': mapping})
df = df.dropna(subset='text')

<ipython-input-8-0efff7f789c2>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'label': mapping})


In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)

Use a pre-trained tokenizer from Hugging Face to tokenize the text data.

In [ ]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Load a pre-trained model and modify it for binary classification.

In [ ]:
from transformers import AutoModelForSequenceClassification

# Load a pre-trained model and modify it for binary classification
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


Set up the training arguments and train the model.

In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# Train the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: uliana108u (uliana108u-ikea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.126400,0.080885
2,0.047100,0.084961
3,0.019600,0.092207


TrainOutput(global_step=3000, training_loss=0.07690773169199626, metrics={'train_runtime': 2528.1311, 'train_samples_per_second': 18.986, 'train_steps_per_second': 1.187, 'total_flos': 6358435135488000.0, 'train_loss': 0.07690773169199626, 'epoch': 3.0})

Evaluate the model on the test set.

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.0922066941857338, 'eval_runtime': 58.5683, 'eval_samples_per_second': 68.296, 'eval_steps_per_second': 4.269, 'epoch': 3.0}


Save the trained model for future use.

In [ ]:
# Save the model
model.save_pretrained('drive/MyDrive/Job_application/VK_ML_engineer/spam_classifier')#./spam_classifier')
tokenizer.save_pretrained('drive/MyDrive/Job_application/VK_ML_engineer/spam_classifier/spam_classifier')

('drive/MyDrive/Job_application/VK_ML_engineer/spam_classifier/spam_classifier/tokenizer_config.json',
 'drive/MyDrive/Job_application/VK_ML_engineer/spam_classifier/spam_classifier/special_tokens_map.json',
 'drive/MyDrive/Job_application/VK_ML_engineer/spam_classifier/spam_classifier/vocab.txt',
 'drive/MyDrive/Job_application/VK_ML_engineer/spam_classifier/spam_classifier/added_tokens.json',
 'drive/MyDrive/Job_application/VK_ML_engineer/spam_classifier/spam_classifier/tokenizer.json')

You can now use the trained model to classify new emails as spam or not spam.

In [ ]:
def classify_email(email_text):
    # Tokenize the input text
    inputs = tokenizer(email_text, return_tensors="pt", padding=True, truncation=True)

    # Debug: Print device of tokenized inputs
    print(f"Tokenized inputs are on device: {inputs['input_ids'].device}")

    # Move inputs to the same device as the model (GPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Debug: Print device of inputs after moving
    print(f"Inputs after moving to device: {inputs['input_ids'].device}")

    # Perform inference
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)

    # Get predictions
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Return the result
    return "Spam" if predictions.item() == 1 else "Not Spam"

# Example usage
email = "Congratulations! You've won a $1000 Walmart gift card. Click here to claim your prize."
print(classify_email(email))

Tokenized inputs are on device: cpu
Inputs after moving to device: cuda:0
Spam
